In [ ]:
# This script reads the uwnd and vwnd NetCDF files, converts them to 3D numpy arrays, and converts the time variable to day of year (DOY). 
# It then creates an animation of the wind vectors using matplotlib.

# Convert the uwnd and vwnd NetCDF files to 3D numpy arrays and convert the time variable as day of year (DOY) (copied from prepare_ai_ready_data.ipynb)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import netCDF4 as nc
import numpy as np
import datetime

# Open the NetCDF file
file_path = '..\\data\\raw\\reanalyses\\ncep\\uwnd.sfc.2024.nc'
dataset = nc.Dataset(file_path, 'r')

# Extract the uwnd variable
uwnd_var = dataset.variables['uwnd']

# Convert the uwnd variable to a 3D numpy array
uwnd_3d_array = uwnd_var[:]

# Open the vwnd NetCDF file
vwnd_file_path = '..\\data\\raw\\reanalyses\\ncep\\vwnd.sfc.2024.nc'
vwnd_dataset = nc.Dataset(vwnd_file_path, 'r')

# Extract the vwnd variable
vwnd_var = vwnd_dataset.variables['vwnd']

# Convert the vwnd variable to a 3D numpy array
vwnd_3d_array = vwnd_var[:]

# Extract the latitudes and longitudes
latitudes = vwnd_dataset.variables['lat'][:]
longitudes = vwnd_dataset.variables['lon'][:]

# Convert the time variable to day of year (DOY)
time_var = vwnd_dataset.variables['time']
reference_date_str = time_var.units.split('since')[1].strip().split('.')[0]
reference_date = datetime.datetime.strptime(reference_date_str, '%Y-%m-%d %H:%M:%S')
doy = [(reference_date + datetime.timedelta(days=t)).timetuple().tm_yday for t in time_var[:]]

# Ensure the DOY values are within the valid range
doy = np.array(doy)
valid_indices = (doy >= 1) & (doy <= 365)
doy = doy[valid_indices]
uwnd_3d_array = uwnd_3d_array[valid_indices]
vwnd_3d_array = vwnd_3d_array[valid_indices]

# Print the shape of the uwnd and vwnd arrays
print(uwnd_3d_array.shape)
print(vwnd_3d_array.shape)

# Close the NetCDF files
dataset.close()
vwnd_dataset.close()

# Print the minimum and maximum values of the lat and lon in each array
print(f"uwnd_3d_array lat min: {latitudes.min()}, lat max: {latitudes.max()}")
print(f"uwnd_3d_array lon min: {longitudes.min()}, lon max: {longitudes.max()}")

print(f"vwnd_3d_array lat min: {latitudes.min()}, lat max: {latitudes.max()}")
print(f"vwnd_3d_array lon min: {longitudes.min()}, lon max: {longitudes.max()}")

# Create an animation of the wind vectors using matplotlib
# Assuming uwnd_3d_array, vwnd_3d_array, latitudes, and longitudes are already defined
# uwnd_3d_array and vwnd_3d_array should be 3D arrays with shape (time, lat, lon)
# latitudes and longitudes should be 1D arrays with the same length as the second and third dimensions of uwnd_3d_array and vwnd_3d_array

# Set the animation size limit to unlimited
plt.rcParams['animation.embed_limit'] = 100.0

# Define the time variable (assuming it is a 1D array with the same length as the first dimension of uwnd_3d_array and vwnd_3d_array)
time = np.arange(uwnd_3d_array.shape[0])

# Create a figure and axis
fig, ax = plt.subplots()

# Set up the plot limits and labels
ax.set_xlim(longitudes.min(), longitudes.max())
ax.set_ylim(latitudes.min(), latitudes.max())
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Initialize the plot with an empty quiver plot
quiver = ax.quiver([], [], [], [])

# Function to update the plot for each frame
def update(frame):
    global quiver
    quiver.remove()
    quiver = ax.quiver(longitudes, latitudes, uwnd_3d_array[frame], vwnd_3d_array[frame])
    ax.set_title(f"Day of Year: {frame + 1}")  # Display the day of year
    return quiver

# Create the animation and assign it to a variable to prevent deletion
ani = animation.FuncAnimation(fig, update, frames=len(time), blit=False)

# Display the animation inline in Jupyter Notebook or VS Code
HTML(ani.to_jshtml())